# Problem description

This notebook uses CP Optimizer to solve Sudoku problems.

# Reading the data

In [1]:
# This is the grid of the so-called "world's hardest sudoku" by Arto Inkala, 2012.
Grid = [
 [8,0,0,0,0,0,0,0,0],  
 [0,0,3,6,0,0,0,0,0], 
 [0,7,0,0,9,0,2,0,0], 
 [0,5,0,0,0,7,0,0,0], 
 [0,0,0,0,4,5,7,0,0], 
 [0,0,0,1,0,0,0,3,0], 
 [0,0,1,0,0,0,0,6,8], 
 [0,0,8,5,0,0,0,1,0],
 [0,9,0,0,0,0,4,0,0], 
]

# Modeling the problem with CP Optimizer

In [2]:
# Import Constraint Programming modelization functions
from docplex.cp.model import *

# Create model object
model = CpoModel()

# Decision variables
x = [ [integer_var(min=1,max=9,name='x'+str(r)+str(c)) for c in range(9)] for r in range(9) ]

# Constraints: input grid values
for r in range(9):
    for c in range(9):
        if Grid[r][c]!=0:
            model.add(x[r][c] == Grid[r][c])

# Constraints: different values in each row
for r in range(9):
    model.add(all_diff([x[r][c] for c in range(9)]))

# Constraints: different values in each column    
for c in range(9):
    model.add(all_diff([x[r][c] for r in range(9)]))

# Constraints: different values in each sub-square 
for sr in range(3):
    for sc in range(3):
        model.add(all_diff([x[r][c] for r in range(3*sr,3*sr+3) for c in range(3*sc,3*sc+3)]))

# Solving the problem with CP Optimizer automatic search

The model can be solved by calling CP Optimizer's automatic search:

In [3]:
# Solve the model
sol = model.solve(trace_log=True)

 ! ----------------------------------------------------------------------------
 ! Satisfiability problem - 81 variables, 48 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 122.9 (before), 122.9 (after)
 !  . Memory usage      : 337.4 kB (before), 337.4 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
                     1000         22    1   F     6  = x32
                     1000         32    2   F     3  = x74
 *                    371  0.01s        3         8  = x14
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Number of branches     : 2371
 ! Number of fails        : 1157
 ! Total memory usage     : 2.1 MB (2.0 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.01s (0.01s engine + 0.0

# Displaying the input grid and the solution

In [4]:
print("Input grid:")
for r in range(9):
    for c in range(9):
        if Grid[r][c]==0:
          print('_ ', end='')
        else:
          print(str(Grid[r][c])+' ', end='')
    print()

print()
print("Solution:")
for r in range(9):
    for c in range(9):
        print(str(sol.get_var_solution(x[r][c]).get_value())+' ', end='')
    print()

Input grid:
8 _ _ _ _ _ _ _ _ 
_ _ 3 6 _ _ _ _ _ 
_ 7 _ _ 9 _ 2 _ _ 
_ 5 _ _ _ 7 _ _ _ 
_ _ _ _ 4 5 7 _ _ 
_ _ _ 1 _ _ _ 3 _ 
_ _ 1 _ _ _ _ 6 8 
_ _ 8 5 _ _ _ 1 _ 
_ 9 _ _ _ _ 4 _ _ 

Solution:
8 1 2 7 5 3 6 4 9 
9 4 3 6 8 2 1 7 5 
6 7 5 4 9 1 2 8 3 
1 5 4 2 3 7 8 9 6 
3 6 9 8 4 5 7 2 1 
2 8 7 1 6 9 5 3 4 
5 2 1 9 7 4 3 6 8 
4 3 8 5 2 6 9 1 7 
7 9 6 3 1 8 4 5 2 
